In [1]:
import numpy as np

from scipy.stats import pearsonr
from scipy.stats import spearmanr
from sklearn.metrics.pairwise import cosine_similarity
from scipy import spatial
from scipy import stats

import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.autograd as autograd
import torch.optim as optim
import torch.nn.functional as F
torch.manual_seed(1)

import time
import datetime

from collections import defaultdict
from collections import Counter

from random import randint
import pickle
import nltk
import string
puncs = set(string.punctuation)

In [2]:
embedding_dim = 100
batch_size = 50
min_count = 2

In [17]:
# dataset = 'europarl/training.en'
# label = '_europarl_en'

dataset = 'europarl/training.fr'
label = '_europarl_fr'

# dataset = 'hansards/training.en'
# label = '_hansard_en'

# dataset = 'hansards/training.fr'
# label = '_hansard_fr'

# dataset = 'wa/dev.en'
# label = '_dev'

#read the files
with open(dataset) as f:
    sentences = [l.strip() for l in f.readlines()]

#get all the tokens from the corpus

tokens_list = []
sentence_list = []
for s in sentences:
    split_sent = s.split()
    sentence = []
    for w in split_sent:
        
        tokens_list.append(w)
        sentence.append(w)
#         #filter stopwords
#         if w not in stopwords:
#             tokens_list.append(w)
#             sentence.append(w)
    
    sentence_list.append(sentence)
    
tokens = list(sorted(set(tokens_list)))
print('Vocabulary size', len(tokens))

count_tokens = Counter(tokens_list)

#words appearing fewer than this are not considered words or contexts
#subsample frequent words

temp_sentence_list = []

for s in range(len(sentence_list)):
    temp_sentence_list.append(sentence_list[s])

#Find the infrequent words
for s in range(len(temp_sentence_list)):
    for w in range(len(temp_sentence_list[s])):
        word = temp_sentence_list[s][w]
        if count_tokens[word] < min_count:
            sentence_list[s][w] = '<unk>'
            
            #remove the infrequent words from tokens
            tokens.remove(word)

#we prune rare words altogether

#tokens.append('<unk>')
vocab_size = len(tokens)
print('After rare word pruning', vocab_size)
#remove the infrequent words from the sentences
for s in sentence_list:
    s[:] = [w for w in s if w != '<unk>' and w not in puncs]

flat_token_list = []

for s in sentence_list:
    for w in s:
        flat_token_list.append(w)

Vocabulary size 161769
After rare word pruning 100797


In [18]:
unigram_counts = Counter(flat_token_list)

unigram_probs = defaultdict(float) 

for t in unigram_counts:
    unigram_probs[t] = unigram_counts[t]/len(flat_token_list)

pickle.dump(unigram_probs, open('unigram_probs' + label + '.p', 'wb'))

unigram_probs = pickle.load(open('unigram_probs' + label + '.p', 'rb'))

In [19]:
tokens = list(set(flat_token_list))

#default dictionary key:id value:token
id2tokens = defaultdict(str)
                        
for i in range(len(tokens)):
    id2tokens[i] = tokens[i]
    
#default dictionary key:token value:id
tokens2id = defaultdict(int)

for ind in id2tokens:
    tokens2id[id2tokens[ind]] = ind
    
vocabulary_size = len(tokens2id)
print(vocabulary_size)

100770


In [20]:
#convert dataset to ids
corpus2id = []

for s in sentence_list:
    
    sentence2id = []
    
    for w in s:
        word_id = tokens2id[w]
        sentence2id.append(word_id)
        
    corpus2id.append(sentence2id)
        

In [21]:
with open('corpus2id' + label + '.pickle', 'wb') as f:
    pickle.dump(corpus2id, f)

In [15]:
with open('corpus2id' + label + '.pickle', 'rb') as f:
    corpus2id = pickle.load(f)

In [16]:
corpus2id[0]

[24564, 21978]